# Read Data fram and cleaning the dataframe

In [1]:
!pip install prettytable

  Obtaining dependency information for prettytable from https://files.pythonhosted.org/packages/25/1e/4c284713b092ec384fad4399452f43f6446ad9aabc9c0b3c3c0920cc53b6/prettytable-3.8.0-py3-none-any.whl.metadata


In [1]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [3]:
df = pd.read_csv('DataSet/IMDB-Dataset-65k-movie-review.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'DataSet/IMDB-Dataset-65k-movie-review.csv'

In [ ]:
nltk.download('punkt')

In [ ]:
from nltk.corpus import stopwords
#Customize stopword as per data
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
new_stopwords = ["would","shall","could","might"]
stop_words.extend(new_stopwords)
stop_words.remove("not")
stop_words=set(stop_words)
print(stop_words)

In [8]:
'''-----------------------------Data Cleaning and Preprocessing pipeline----------------------------------'''

#Removing special character
def remove_special_character(content):
    return re.sub('\W+',' ', content )#re.sub('\[[^&@#!]]*\]', '', content)

# Removing URL's
def remove_url(content):
    return re.sub(r'http\S+', '', content)

#Removing the stopwords from text
def remove_stopwords(content):
    clean_data = []
    for i in content.split():
        if i.strip().lower() not in stop_words and i.strip().lower().isalpha():
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)

# Expansion of english contractions
def contraction_expansion(content):
    content = re.sub(r"won\'t", "would not", content)
    content = re.sub(r"can\'t", "can not", content)
    content = re.sub(r"don\'t", "do not", content)
    content = re.sub(r"shouldn\'t", "should not", content)
    content = re.sub(r"needn\'t", "need not", content)
    content = re.sub(r"hasn\'t", "has not", content)
    content = re.sub(r"haven\'t", "have not", content)
    content = re.sub(r"weren\'t", "were not", content)
    content = re.sub(r"mightn\'t", "might not", content)
    content = re.sub(r"didn\'t", "did not", content)
    content = re.sub(r"n\'t", " not", content)
    content = re.sub(r"\'re", " are", content)
    content = re.sub(r"\'s", " is", content)
    content = re.sub(r"\'d", " would", content)
    content = re.sub(r"\'ll", " will", content)
    content = re.sub(r"\'t", " not", content)
    content = re.sub(r"\'ve", " have", content)
    content = re.sub(r"\'m", " am", content)
    return content

#Data preprocessing
def data_cleaning(content):
    content = contraction_expansion(content)
    content = remove_special_character(content)
    content = remove_url(content)
    
    content = remove_stopwords(content)    
    return content


In [10]:
df['Reviews_clean'] = df['Reviews'].apply(data_cleaning)

# Create the Labels for the Training Purpose

In [14]:
df['Label'] = df['Ratings'].apply(lambda x: '1' if x >= 7 else ('0' if x<=4 else '2'))
#Removing 
df=df[df.Label<'2']
data=df[['Reviews_clean','Label']]
print(data['Label'].value_counts())
    
print("0 --> ❌")
print("1 --> ✔️")

0    60000
1    60000
Name: Label, dtype: int64
0 --> ❌
1 --> ✔️


# Lemimatization class

In [24]:
#Importing dependencies for feature engineering 
import nltk
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [15]:
# lemmatization of word 
class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    def __call__(self, reviews):
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]

# Vectoization with Count Vectorizer and TDIDF Vectorizer with Unigram

In [26]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from string import punctuation
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from prettytable import PrettyTable
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

## 1. Unigram 

In [16]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

C:\Users\malan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [18]:
unigram_feature = countvect.get_feature_names_out()
print(unigram_feature)

['able' 'absolutely' 'act' 'acting' 'action' 'actor' 'actress' 'actually'
 'add' 'adult' 'age' 'alien' 'almost' 'along' 'already' 'also' 'although'
 'always' 'amazing' 'american' 'annoying' 'another' 'anyone' 'anything'
 'anyway' 'around' 'art' 'attempt' 'attention' 'audience' 'average' 'away'
 'awful' 'b' 'back' 'bad' 'based' 'beautiful' 'become' 'becomes' 'begin'
 'beginning' 'behind' 'believe' 'best' 'better' 'big' 'bit' 'black'
 'blood' 'body' 'book' 'boring' 'boy' 'brilliant' 'brother' 'budget'
 'call' 'called' 'came' 'camera' 'car' 'care' 'case' 'cast' 'certainly'
 'cgi' 'chance' 'change' 'character' 'cheap' 'child' 'cinema'
 'cinematography' 'classic' 'close' 'come' 'comedy' 'comic' 'coming'
 'complete' 'completely' 'cool' 'couple' 'course' 'crap' 'creature'
 'credit' 'cut' 'dark' 'daughter' 'day' 'dead' 'death' 'decent'
 'definitely' 'despite' 'dialogue' 'different' 'directed' 'direction'
 'director' 'disappointed' 'done' 'drama' 'due' 'dvd' 'editing' 'effect'
 'effort' 'either

## Feature Importance with logistic Regression And Count Vectorizer With Unigra

### 1. for count vect

In [71]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+----------------+------------------------+
|    Feature     |         Score          |
+----------------+------------------------+
|      able      |  0.10570781385867273   |
|   absolutely   |  0.05570466948922224   |
|      act       |  -0.15740554664603085  |
|     acting     |  -0.1688379094581172   |
|     action     |  0.26539310547138634   |
|     actor      |  -0.1288103028807266   |
|    actress     |  -0.1440053058432976   |
|    actually    |  -0.05204574383886588  |
|      add       |  0.026808778810200575  |
|     adult      |  0.056856340675823534  |
|      age       |  0.09923486175892182   |
|     alien      |  -0.07249914939651907  |
|     almost     |  -0.01568711024849215  |
|     along      |  0.12144728279926087   |
|    already     |  -0.20602067823732104  |
|      also      |  0.09768229838483933   |
|    although    |  0.19548648673092667   |
|     always     |   0.2000140789614552   |
|    amazing     |   0.8387171521997259   |
|    american    |  0.0494103446

### 2. for tfidf vect

In [75]:
# extract the numeric value 
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(tfidfvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=100:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+----------------+------------------------+
|    Feature     |         Score          |
+----------------+------------------------+
|      able      |   0.4028810644080798   |
|   absolutely   |  0.49909859674535356   |
|      act       |   -1.050585784200308   |
|     acting     |  -1.6671911644911481   |
|     action     |   2.618420837134261    |
|     actor      |   -1.360170533527896   |
|    actress     |  -0.9998074480374615   |
|    actually    |  -0.49036442484313175  |
|      add       |   0.1377365293961775   |
|     adult      |   0.6910454851099983   |
|      age       |   0.7574690106275593   |
|     alien      |  -0.3387842778817132   |
|     almost     |  -0.3112617850251709   |
|     along      |   0.7474949775099915   |
|    already     |  -1.1593649053620372   |
|      also      |   1.2210120918337648   |
|    although    |   1.2215664980730545   |
|     always     |   1.7116604136402493   |
|    amazing     |   4.793095426900631    |
|    american    |  0.4212764149

## 2. Unigram 

##  Vectoization with Count Vectorizer and TDIDF Vectorizer with Bigram

In [19]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

C:\Users\malan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 1. for count vect

In [20]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+-----------------------+-----------------------+
|        Feature        |         Score         |
+-----------------------+-----------------------+
|       acting bad      |  -1.3586529462787338  |
|      acting good      |   0.6561720782088444  |
|       acting not      |  -0.4628174124851165  |
|      action film      |   0.3693835705647081  |
|      action movie     |  0.29230365454807755  |
|     action packed     |   1.3352284037408662  |
|      action scene     |  0.17571226600762158  |
|    action sequence    |  0.09959663106955677  |
|     actor actress     |  -0.12649253274145802 |
|       actor not       |  -0.38921556342368746 |
|      almost every     |  -0.1323418562333905  |
|       along way       |  0.49420288262635326  |
|        also not       |  -0.13838425047091674 |
|      although not     |   0.5002494318854408  |
|     anything else     |   -0.322339743919431  |
|        b movie        |  -0.09063290278250238 |
|       bad acting      |   -1.060114060567707  |


### 2. for tfidf vect

In [21]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(tfidfvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+-----------------------+-----------------------+
|        Feature        |         Score         |
+-----------------------+-----------------------+
|       acting bad      |  -2.9863365211987936  |
|      acting good      |   1.4458927752755837  |
|       acting not      |  -1.2650693460392226  |
|      action film      |   1.2240429081904436  |
|      action movie     |   0.8475644256942594  |
|     action packed     |   3.013997148922698   |
|      action scene     |   0.4708510637207139  |
|    action sequence    |   0.2870261005978393  |
|     actor actress     |  -0.2628000358579053  |
|       actor not       |  -0.9284315858588352  |
|      almost every     |  -0.29284895695094537 |
|       along way       |   1.0847916777914384  |
|        also not       |  -0.2888818948387973  |
|      although not     |   1.106948458461302   |
|     anything else     |  -0.7956983401427188  |
|        b movie        |   -0.420759268066691  |
|       bad acting      |   -2.798863577346767  |


## 3. Trigraph

##  Vectoization with Count Vectorizer and TDIDF Vectorizer with Trigram

In [10]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(3,3), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(3,3),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

### 1. for count vect

In [12]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+---------------------------+-----------------------+
|          Feature          |         Score         |
+---------------------------+-----------------------+
|       acting not bad      |   -0.415380235831366  |
|      acting not good      |   -1.061344600576712  |
|      acting not great     |  -0.46845504439397023 |
|     acting pretty good    |   0.9606468885951975  |
|      acting top notch     |   1.8053803251903233  |
|      action movie not     |   1.053903301611979   |
|      action set piece     |   1.2150841421254754  |
|     action take place     |  0.04178974004676074  |
|       actor good job      |   0.6022263423652255  |
|    actually pretty good   |   0.3230995327465889  |
|    actually quite good    |  -0.1646734748757942  |
|     almost every scene    |  -0.32065374013820785 |
|        b movie not        | -0.048632905798628404 |
|       bad acting bad      |   -2.328866264693395  |
|        bad bad bad        |   -1.76371650822253   |
|        bad guy not        

### 2. for tfidf vect

In [13]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+---------------------------+------------------------+
|          Feature          |         Score          |
+---------------------------+------------------------+
|       acting not bad      |  -0.5736683360148773   |
|      acting not good      |  -1.3111959907461346   |
|      acting not great     |  -0.6611341432138254   |
|     acting pretty good    |   1.0995715755857955   |
|      acting top notch     |   2.071523010019828    |
|      action movie not     |   1.2373035768326353   |
|      action set piece     |   1.2431258924385509   |
|     action take place     |   0.0901920750281822   |
|       actor good job      |   0.6809761822271533   |
|    actually pretty good   |   0.3527855416953325   |
|    actually quite good    |  -0.14271747094013387  |
|     almost every scene    |  -0.3829174112938804   |
|        b movie not        |  -0.09385974780309414  |
|       bad acting bad      |   -2.858295581309548   |
|        bad bad bad        |   -2.588573137603917   |
|        b

In [15]:
from sklearn.feature_selection import chi2
import numpy as np
N = 5000
Number = 1
featureselection = PrettyTable(["Unigram", "Bigram","Trigram"])
for category in train['Label'].unique():
    features_chi2 = chi2(x_train_tfidf, train['Label'] == category)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidfvect.get_feature_names_out())[indices]
    unigrams = [x for x in feature_names if len(x.split(' ')) == 1]
    bigrams = [x for x in feature_names if len(x.split(' ')) == 2]
    trigrams = [x for x in feature_names if len(x.split(' ')) == 3]
    print("%s. %s :" % (Number,category))
    print("\t# Unigrams :\n\t. %s" %('\n\t. '.join(unigrams[-N:])))
    print("\t# Bigrams :\n\t. %s" %('\n\t. '.join(bigrams[-N:])))
    print("\t# Trigrams :\n\t. %s" %('\n\t. '.join(trigrams[-N:])))
    Number += 1

1. 1 :
	# Unigrams :
	. 
	# Bigrams :
	. 
	# Trigrams :
	. http www imdb
	. movie not going
	. not help think
	. not really know
	. movie not bad
	. not bad film
	. www imdb com
	. really not like
	. movie low budget
	. minute running time
	. story line not
	. not whole lot
	. without doubt one
	. best part film
	. film like one
	. can not say
	. film can not
	. movie not think
	. not think anyone
	. first one not
	. high school musical
	. not seem like
	. movie one not
	. ever seen film
	. film feel like
	. claude van damme
	. know last summer
	. hand held camera
	. movie not get
	. not think ever
	. night living dead
	. movie not see
	. not think much
	. found footage film
	. special effect movie
	. cuba gooding jr
	. make movie like
	. ever made not
	. not great not
	. one movie not
	. not care much
	. first two movie
	. movie movie not
	. want see movie
	. sci fi movie
	. film much better
	. really not know
	. not half bad
	. movie take place
	. jean claude van
	. saturday night li